<a href="https://colab.research.google.com/github/ale1995co/Final-Capstone/blob/master/ALS_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install Java, Spark, Findspark and PySpark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

!pip install -q findspark
!pip install pyspark

# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import time

# spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.mllib.recommendation import ALS
from pyspark.sql import Row,SQLContext, SparkSession

# data science imports
import math
import numpy as np
import pandas as pd

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
USERS_PATH = "/content/gdrive/My Drive/Colab Notebooks/Colab Datasets/Final Capstone/users_cleaned.csv" 
ANIME_PATH = "/content/gdrive/My Drive/Colab Notebooks/Colab Datasets/Final Capstone/anime_cleaned.csv"
SCORES_PATH = "/content/gdrive/My Drive/Colab Notebooks/Colab Datasets/Final Capstone/animelists_cleaned.csv"
APP_NAME = "Anime Recommender"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
sc = spark.sparkContext

# Context 

The csvs that are use for pyspark are the clean ones. The creator of the dataset omitted rows that has rows the best of his ability. I tried running the filtered ones but it was giving me errors through pyspark.

# Load Data

In [0]:
sqlContext = SQLContext(sc)
usersdf = spark.read.load(os.path.join(USERS_PATH), format='csv', header=True, inferSchema=True)
animedf = spark.read.load(os.path.join(ANIME_PATH), format='csv', header=True, inferSchema=True)
scoresdf =spark.read.load(os.path.join(SCORES_PATH), format='csv', header=True, inferSchema=True)

In [11]:
usersdf.show(5)

+--------------+-------+-------------+--------------+-----------+------------+----------------+------------------------+------+-------------------+-------------------+-----------+-------------------+-------------------+----------------+---------------+--------------+
|      username|user_id|user_watching|user_completed|user_onhold|user_dropped|user_plantowatch|user_days_spent_watching|gender|           location|         birth_date|access_rank|          join_date|        last_online|stats_mean_score|stats_rewatched|stats_episodes|
+--------------+-------+-------------+--------------+-----------+------------+----------------+------------------------+------+-------------------+-------------------+-----------+-------------------+-------------------+----------------+---------------+--------------+
|      karthiga|2255153|            3|            49|          1|           0|               0|       55.09166666666667|Female|    Chennai, India |1990-04-29 00:00:00|       null|2013-03-03 00:00:

In [12]:
animedf.show(5)

+--------+-------------------+--------------------+----------------------+--------------------+--------------------+----+--------+--------+---------------+------+--------------------+--------------------+---------------+--------------------+-----+---------+------+----------+--------+---------+--------------------+-----------+------------------+--------------------+--------------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|anime_id|              title|       title_english|        title_japanese|      title_synonyms|           image_url|type|  source|episodes|         status|airing|        aired_string|               aired|       duration|              rating|score|scored_by|  rank|popularity| members|favorites|          background|  premiered|         broadcast|             related|            producer|        licensor|          studio|               genre|       opening_theme|        

In [14]:
scoresdf.show(5)

+--------+--------+-------------------+-------------+--------------+--------+---------+-------------+----------------+-------------------+-------+
|username|anime_id|my_watched_episodes|my_start_date|my_finish_date|my_score|my_status|my_rewatching|my_rewatching_ep|    my_last_updated|my_tags|
+--------+--------+-------------------+-------------+--------------+--------+---------+-------------+----------------+-------------------+-------+
|karthiga|      21|                586|   0000-00-00|    0000-00-00|       9|        1|         null|               0|2013-03-03 10:52:53|   null|
|karthiga|      59|                 26|   0000-00-00|    0000-00-00|       7|        2|         null|               0|2013-03-10 13:54:51|   null|
|karthiga|      74|                 26|   0000-00-00|    0000-00-00|       7|        2|         null|               0|2013-04-27 16:43:35|   null|
|karthiga|     120|                 26|   0000-00-00|    0000-00-00|       7|        2|         null|               0|

In [15]:
scoresdf.dtypes

[('username', 'string'),
 ('anime_id', 'string'),
 ('my_watched_episodes', 'string'),
 ('my_start_date', 'string'),
 ('my_finish_date', 'string'),
 ('my_score', 'string'),
 ('my_status', 'string'),
 ('my_rewatching', 'string'),
 ('my_rewatching_ep', 'string'),
 ('my_last_updated', 'string'),
 ('my_tags', 'string')]

In [16]:
scoresdf = scoresdf.drop("my_watched_episodes", "my_start_date", "my_finish_date", "my_rewatching", 
                         "my_last_updated", "my_tags", "my_status", "my_rewatching_ep" )
scoresdf.show(5)

+--------+--------+--------+
|username|anime_id|my_score|
+--------+--------+--------+
|karthiga|      21|       9|
|karthiga|      59|       7|
|karthiga|      74|       7|
|karthiga|     120|       7|
|karthiga|     178|       7|
+--------+--------+--------+
only showing top 5 rows



The ALS model only use three features. Breaking this dataframe to down organize it.

## Questions I like to answer using pyspark tools:

What are the scores?
What is the total number of users in the data sets?
What is the total number of anime in the data sets?
How many anime are rated by users? List how many anime not rated yet?

In [17]:
scoresdf.dtypes

[('username', 'string'), ('anime_id', 'string'), ('my_score', 'string')]

In [0]:
from pyspark.sql.types import IntegerType

In [0]:
scoresdf = scoresdf.withColumn("my_score", scoresdf["my_score"].cast(IntegerType()))

Changing the my_score to integers

What are the scores?

In [21]:
scoresdf.groupby("my_score")\
        .count()\
        .show()

+--------+--------+
|my_score|   count|
+--------+--------+
|    null|  100962|
|       1|  103177|
|       6| 2128502|
|       3|  223202|
|       5| 1085660|
|       9| 3443674|
|      17|       1|
|       4|  480871|
|       8| 4834595|
|       7| 4234726|
|      10| 2507404|
|       2|  130314|
|    2017|       1|
|       0|12111905|
+--------+--------+



In [62]:
# null, 17, 0, 2017 doesn't exist in the rating system. 
scoresdf = scoresdf.na.drop()
scoresdf = scoresdf.filter((scoresdf["my_score"] != 0) & (scoresdf["my_score"] != 2017) & (scoresdf["my_score"] != 17 ))
scoresdf.groupby("my_score")\
        .count()\
        .show()

+--------+-------+
|my_score|  count|
+--------+-------+
|       1| 103177|
|       6|2128502|
|       3| 223202|
|       5|1085660|
|       9|3443674|
|       4| 480871|
|       8|4834595|
|       7|4234726|
|      10|2507404|
|       2| 130314|
+--------+-------+



What is the total number of users in the data sets?

In [63]:
tmp = scoresdf.select('username').distinct().count()
print('We have a total of {} distinct users in the data sets'.format(tmp))

We have a total of 106402 distinct users in the data sets


How many anime are rated by users? List how many anime not rated yet?

In [64]:
tmp1 = animedf.select('anime_id').distinct().count()
tmp2 = scoresdf.select('anime_id').distinct().count()
print('We have a total of {} distinct anime that are rated by users in ratings table'.format(tmp2))
print('We have {} anime that are not rated yet'.format(tmp1-tmp2))

We have a total of 6598 distinct anime that are rated by users in ratings table
We have 70 anime that are not rated yet


In [65]:
animedf.createOrReplaceTempView("animedf")
scoresdf.createOrReplaceTempView("scoresdf")
print('List movies that are not rated yet: ')
# SQL query (NOTE: WHERE ... NOT IN ... == ... LEFT JOIN ... WHERE ... IS NULL)
# Approach 1
spark.sql(
    "SELECT anime_id, title "
    "FROM animedf "
    "WHERE anime_id NOT IN (SELECT distinct(anime_id) FROM scoresdf)"
).show(10)

List movies that are not rated yet: 
+--------+--------------------+
|anime_id|               title|
+--------+--------------------+
|    7639|Shounen Santa no ...|
|   33310|Peach Command Shi...|
|   28119|Kuma no Minakuro ...|
|   33484|          Shiroi Zou|
|   31020|Norasco: Cinema P...|
|   34091|    Norasco Specials|
|   32663|Tama & Friends: U...|
|   37187| Kuiba Yao Xia Zhuan|
|   20001|     Mouretsu Atarou|
|   23537|  Two Down Full Base|
+--------+--------------------+
only showing top 10 rows



Converting the two dataframes of user-id, anime_id, scores

In [23]:
left_join = usersdf.join(scoresdf, usersdf.username == scoresdf.username,how='left') # Could also use 'left_outer'
left_join.show()

+--------+-------+-------------+--------------+-----------+------------+----------------+------------------------+------+--------------------+-------------------+-----------+-------------------+-------------------+----------------+---------------+--------------+--------+--------+--------+
|username|user_id|user_watching|user_completed|user_onhold|user_dropped|user_plantowatch|user_days_spent_watching|gender|            location|         birth_date|access_rank|          join_date|        last_online|stats_mean_score|stats_rewatched|stats_episodes|username|anime_id|my_score|
+--------+-------+-------------+--------------+-----------+------------+----------------+------------------------+------+--------------------+-------------------+-----------+-------------------+-------------------+----------------+---------------+--------------+--------+--------+--------+
| -Himeko|2156961|           13|           164|          0|           0|               0|       34.59513888888889|Female|Buenos Ai

In [0]:
left_join= left_join.select("user_id", "my_score", "anime_id" )
#Only need three features for the ALS model. 

In [26]:
left_join.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- my_score: integer (nullable = true)
 |-- anime_id: string (nullable = true)



# ALS Model

ALS is one of the low rank matrix approximation algorithms for collaborative filtering. ALS decomposes user-item matrix into two low rank matrixes: user matrix and item matrix. In collaborative filtering, users and products are described by a small set of latent factors that can be used to predict missing entries. And ALS algorithm learns these latent factors by matrix factorization

In [0]:
left_join = left_join.withColumn("anime_id", scoresdf["anime_id"].cast(IntegerType()))

In [28]:
left_join.printSchema

<bound method DataFrame.printSchema of DataFrame[user_id: int, my_score: int, anime_id: int]>

In [0]:
#Split dataset to train and test
train_data, test_data = left_join.randomSplit([0.8, 0.2])

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [67]:
left_join = left_join.na.drop()
#nulls will crash the ALS model
left_join.describe()

DataFrame[summary: string, user_id: string, my_score: string, anime_id: string]

In [0]:
model = ALS(userCol='user_id', itemCol='anime_id', ratingCol='my_score').fit(left_join)

In [41]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', labelCol='my_score', predictionCol='prediction')
print('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions)))

The root mean squared error for our model is: 1.1800121568249533


In [0]:
(trainingRatings, testRatings) = left_join.randomSplit([80.0, 20.0])


In [0]:
als = ALS(userCol='user_id', itemCol='anime_id', ratingCol='my_score')
model = als.fit(trainingRatings)
predictions = model.transform(testRatings)

In [44]:
predictions.toPandas().head()

,user_id,my_score,anime_id,prediction
0,222730,8,148,7.415411
1,46781,7,148,6.964195
2,301644,3,148,5.483258
3,65288,9,148,7.210005
4,19701,8,148,7.050633


In [46]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='my_score', predictionCol='prediction')
print('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions)))

The root mean squared error for our model is: nan


nan happens when the model can't predict value that they don't have data on.

Replace predicted NaN values with the average rating and evaluate the model:

In [49]:
avgRatings = left_join.select('my_score').groupBy().avg().first()[0]
print ('The average rating in the dataset is: {}'.format(avgRatings))

evaluator = RegressionEvaluator(metricName='rmse', labelCol='my_score', predictionCol='prediction')
print ('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.fill(avgRatings))))

The average rating in the dataset is: 7.59139401605195
The root mean squared error for our model is: 1.2258500119320432


Now exclude predicted NaN values and evaluate the model:

In [51]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='my_score', predictionCol='prediction')
print ('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.drop())))

The root mean squared error for our model is: 1.2256871520044708


Now we can run the model to test the recommendations.

In [0]:
from pyspark.sql.functions import lit

def recommendMovies(model, user, nbRecommendations):
    # Create a Spark DataFrame with the specified user and all the anime listed in the ratings DataFrame
    dataSet = left_join.select('anime_id').distinct().withColumn('user_id', lit(user))

    # Create a Spark DataFrame with the anime that have already been rated by this user
    moviesAlreadyRated = left_join.filter(left_join.user_id == user).select('anime_id', 'user_id')

    # Apply the recommender system to the data set without the already rated anime to predict ratings
    predictions = model.transform(dataSet.subtract(moviesAlreadyRated)).dropna().orderBy('prediction', ascending=False).limit(nbRecommendations).select('anime_id', 'prediction')

    # Join with the movies DataFrame to get the anime titles and genres
    recommendations = predictions.join(animedf, predictions.anime_id == animedf.anime_id).select(predictions.anime_id, animedf.title, animedf.genre, predictions.prediction, animedf.premiered)

#     recommendations.show(truncate=False)
    return recommendations

In [61]:
print('Recommendations for user 3924291:')
recommendMovies(model, 3924291, 10).toPandas()

Recommendations for user 133:


,anime_id,title,genre,prediction,premiered
0,35478,Neko no Robu,"Comedy, Slice of Life",9.231347,Spring 2017
1,32281,Kimi no Na wa.,"Supernatural, Drama, Romance, School",8.985014,None
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",8.959611,Spring 2015
3,35180,3-gatsu no Lion 2nd Season,"Drama, Game, Seinen, Slice of Life",8.829355,Fall 2017
4,37179,Mao Yu Tao Hua Yuan,"Adventure, Comedy, Fantasy",8.856974,None
5,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",8.836061,Fall 2012
6,30484,Steins;Gate 0,"Sci-Fi, Thriller",8.874432,Spring 2018
7,28851,Koe no Katachi,"Drama, School, Shounen",8.862169,None
8,9969,Gintama&#039;,"Action, Sci-Fi, Comedy, Historical, Parody, Sa...",8.841551,Spring 2011
9,263,Hajime no Ippo,"Comedy, Sports, Drama, Shounen",8.827897,Fall 2000


In [59]:
print('Recommendations for user 303577:')
recommendMovies(model, 303577, 10).toPandas()

Recommendations for user 133:


,anime_id,title,genre,prediction
0,35478,Neko no Robu,"Comedy, Slice of Life",10.291141
1,32281,Kimi no Na wa.,"Supernatural, Drama, Romance, School",9.525883
2,7311,Suzumiya Haruhi no Shoushitsu,"Comedy, Mystery, Romance, School, Sci-Fi, Supe...",9.515804
3,35180,3-gatsu no Lion 2nd Season,"Drama, Game, Seinen, Slice of Life",9.456108
4,5114,Fullmetal Alchemist: Brotherhood,"Action, Military, Adventure, Comedy, Drama, Ma...",9.611381
5,36732,Qin Shi Ming Yue: Tian Xing Jiu Ge,"Action, Historical, Martial Arts, Fantasy",9.529469
6,9253,Steins;Gate,"Thriller, Sci-Fi",9.541064
7,37179,Mao Yu Tao Hua Yuan,"Adventure, Comedy, Fantasy",9.712524
8,24701,Mushishi Zoku Shou 2nd Season,"Adventure, Fantasy, Historical, Mystery, Seine...",9.482000
9,9969,Gintama&#039;,"Action, Sci-Fi, Comedy, Historical, Parody, Sa...",9.474827


The ALS algorthim shows the top ten anime recommendation. When comparing the results from the baseline, the anime varied but both has the same anime with each other. The ALS leans toward anime that are not known but includes popular anime also. While baseline shows mostly popular anime. The RMSE for ALS is 1.22568. While Baselineonly is 1.289960.  

# Conclusion

If I have to use one recommendation system it would be the ALS because it accounts for all data and not samples like the baseline. Also it has a lower RSME than baseline This recomendation system would be great for the website to help recommend the users what anime to watch. Since the system cater towards people who are familar with anime. I can see it run similar like netflix or spotify recommendation system . To go further is to use all the data for the model for baseline model and knn to use it for the full potential.  